In [1]:
#先test产生outputs
from baseline import LanguagePlusImage
from Config import Config
from data import ReferExpressionDataset
from refer import REFER

use_train_dataset = False

cfg = Config()
refer = REFER(cfg, train=True)
refer_val = REFER(cfg, False)

loading dataset into memory...
creating index...
index created.
DONE (t=3.00s)
loading dataset into memory...
creating index...
index created.
DONE (t=0.23s)


In [3]:
dataset = ReferExpressionDataset(cfg, refer, split=False) # 用来训练的
val_dataset = ReferExpressionDataset(cfg, refer_val) # 用来验证的
#test_dataset = ReferExpressionDataset(cfg, refer, split=True, test=True)
test_dataset = ReferExpressionDataset(cfg, refer_val, test=True)
test_train_dataset = ReferExpressionDataset(cfg, refer, test=True,split=False) # 用训练数据测试看看，安慰自己
model = LanguagePlusImage(cfg)

Using cuda
Using cuda
Using cuda
Freeze image network weights


In [4]:
checkpt_file = 'checkpoints/default.mdl.checkpoint119'
model.load_model(checkpt_file)

=> loading checkpoint 'checkpoints/default.mdl.checkpoint119'
=> loaded checkpoint 'checkpoints/default.mdl.checkpoint119' (epoch 119)


In [5]:
if use_train_dataset:
    outputs = model.run_test(test_train_dataset)
else:
    outputs = model.run_test(test_dataset)

Test None: 100%|███████████████████████████████████████████████████████████████████| 5000/5000 [18:25<00:00,  4.52it/s]


In [6]:
import numpy as np

a = np.array(outputs)
np.save('a.npy',a)

In [ ]:
import numpy as np

outputs = np.load('a.npy',allow_pickle=True)
outputs = outputs.tolist()

In [7]:
# 用来产生comprehesion_output文件的
import json

comprehension_val_inputs = []
for output in outputs:
    comprehension_val_input = dict()
    predicted_bounding_boxes = []
    ann_id = output['annID'][0].numpy()[0]
    comprehension_val_input['annotation_id'] = int(ann_id)
    
    
    if use_train_dataset:
        imageID = refer.Anns[str(ann_id)]['image_id']
        region_candidates = refer.Imgs[str(imageID)]['region_candidates']
        bboxes = [region_candidate['bounding_box'] for region_candidate in region_candidates]
        tmp = []
        tmp.append(refer.Anns[str(ann_id )]['bbox'])
        tmp[1:] = bboxes[:]
        bboxes = tmp
    else:
        imageID = refer_val.Anns[str(ann_id)]['image_id']
        region_candidates = refer_val.Imgs[str(imageID)]['region_candidates']
        bboxes = [region_candidate['bounding_box'] for region_candidate in region_candidates]
        tmp = []
        tmp.append(refer_val.Anns[str(ann_id )]['bbox'])
        tmp[1:] = bboxes[:]
        bboxes = tmp
        
    for idx in output['sorted_bboxes_idx']:
        predicted_bounding_boxes.append(bboxes[idx])
    comprehension_val_input['predicted_bounding_boxes'] = predicted_bounding_boxes
    comprehension_val_inputs.append(comprehension_val_input)

if use_train_dataset:
    comprehension_file_name = 'comprehension_train.json'
else:
    comprehension_file_name = 'comprehension.json'
with open(comprehension_file_name, 'w') as f:
    json.dump(comprehension_val_inputs, f)

In [8]:
%matplotlib inline
import sys
import os
import numpy
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

# MAKE Sure that google_refexp_py_lib is in your python libary search path
# before you run API in this toolbox. You can use something as follows:aa
sys.path.append('./google_refexp_py_lib')  # 这句是干什么的
from refexp_eval import RefexpEvalComprehension
from common_utils import draw_bbox

use_train_data = False

# Set coco_data_path and Google Refexp dataset validation set path
if use_train_data:
    refexp_dataset_path = './google_refexp_dataset_release/google_refexp_train_201511_coco_aligned.json'
else:
    refexp_dataset_path = './google_refexp_dataset_release/google_refexp_val_201511_coco_aligned.json'
coco_data_path = '../Google_Refexp_toolbox-master/external/coco/annotations/instances_train2014.json'

In [9]:
# Load Google Refexp dataset and MS COCO dataset (takes some time)
eval_compreh = RefexpEvalComprehension(refexp_dataset_path, coco_data_path)

Loading Google Refexp dataset file for the comprehension task.
loading annotations into memory...
Done (t=14.05s)
creating index...
index created!
Dataset loaded.


In [10]:
# We evalute a sample comprehension task results generated by
# a naive method which outputs the groundtruth bounding boxes
# in the coco image with a random order.  这一块究竟是个什么意思呢？！
if use_train_data:
    pred_results_path = 'comprehension_train.json'
else:
    pred_results_path = 'comprehension.json'

(prec, eval_results) = eval_compreh.evaluate(pred_results_path)

Loading predicted result file for the comprehension task.
The average prec@1 score is 0.499
